In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import display, HTML
from met_explore.models import *
from met_explore.serializers import *
import pandas as pd
import numpy as np
from collections import OrderedDict
import os
import json
import collections
from difflib import SequenceMatcher
from collections import OrderedDict
PROTON = 1.007276


In [ ]:
cwd = os.getcwd()
print (cwd)


In [ ]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
   print (f)

### JSON file in format {sample: {peak_id:intensity, peak_id:intensity...}}

### Number of peak objects to be stored

In [ ]:
with open('67_peak_cmpd_export_1.json') as json_peak_data:
    peak_details = json.load(json_peak_data)
    print (len(peak_details['pid']))

### Dataframe returned from PiMP

In [ ]:
peak_details_df= pd.read_json('67_peak_cmpd_export_1.json')


### Filter on adduct type on accepting M+H and M-H

In [ ]:
selected_adducts = (peak_details_df['adduct']=='M+H') | (peak_details_df['adduct']=='M-H')
f_adducts = peak_details_df[selected_adducts]
display(f_adducts)








#### Write a small assert function for this. - to check we have the correct numbers.

In [ ]:
test = f_adducts['frank_annot'].notnull()
f_annots = f_adducts[test]
display(f_annots.shape)
 #1234 rows with a FrAnk annotation
test2 = f_adducts['identified']=="True"
display(f_adducts[test2].shape)
#380 rows with identified = True
test3 =  (f_adducts['identified']=="True") & (f_adducts['frank_annot'].notnull())
display(f_adducts[test3].shape[0])
# 160 rows have identified and true
# The number of ID and annotated peaks should be:
1234+380-160

### Selected on identified = true and/or frank annotations exist.
#### (have selected_df as a member variable?


In [ ]:
with_annot = (f_adducts['frank_annot'].notnull()) | (f_adducts['identified']=='True')
selected_df = f_adducts[with_annot]


In [ ]:
selected_df = add_neutral_masses(selected_df)
display(selected_df)


### Looking at the df in two sections - first the Identified rows.

In [ ]:
headers = list(selected_df.columns.values)
final_id_df = pd.DataFrame(columns=headers)
print("final id df")
display(final_id_df)
all_id_df = selected_df[selected_df.identified=='True']

id_df = get_id_cmpds(all_id_df)

In [ ]:
single_sid_df = selected_df[selected_df.sec_id==2590]
cmpd_id = sid_df[sid_df.db=="stds_db"]['cmpd_id'].values
print (cmpd_id)

## Main method (atm)

In [ ]:
headers = list(selected_df.columns.values)
final_df = pd.DataFrame(columns=headers)
unique_sec_ids = selected_df['sec_id'].unique()

#unique_sec_ids = [2212,3976]

for sid in unique_sec_ids:
    #Collect a single sec_id into a DF
    sid_df = selected_df[selected_df.sec_id==sid]
    print("The single SID DF is")
    display(sid_df)
    #If the peak has an identified compound then keep that
    identified_df = sid_df[sid_df.identified=='True']
    print ("The identified df is: ")
    display(identified_df)    
    new_row = None
    #If some of the rows have compounds that have identified=True
    if not identified_df.empty:
                  
        #Check if there are more than one standard compounds for this sid
        standard_cmpds = sid_df[sid_df.db=='stds_db']
        num_std_cmpds = standard_cmpds.shape[0]
        
        # If there is only one standard compound add this to the final DF and collect identifiers. 
        if (num_std_cmpds == 1):
            
            print ("we have only one standard compound")
            cmpd_id = sid_df[sid_df.db=="stds_db"]['cmpd_id'].values[0]
            new_row = get_peak_by_cmpd_id(sid_df, cmpd_id)            
        
        #Here we have the senario that more that 1 standard compound has been identified and we 
        #want to select a standard compound if possible 
        if (num_std_cmpds > 1):
            print ("the number of standard compounds for sid is", sid, "is", num_std_cmpds)
            new_row = select_standard_cmpd(sid_df, standard_cmpds)
            

        #If a new_row has been returned for this SID - add it to the final_df
        if new_row is not None:
            
            print("we are adding the row for sid", sid)
            display(pd.DataFrame(new_row).T)
            final_df = final_df.append(new_row)  

        #If the new_row has not been determined for this SID
        else:
            
            unique_cmpd_ids = sid_df['cmpd_id'].unique()

            #For each unique compound id add a row to the final df, this will produce duplicates for later    
            for ucid in unique_cmpd_ids:
                
                new_row = get_peak_by_cmpd_id(sid_df, ucid)   
                print("we are adding the row: for sid", sid)
                display(pd.DataFrame(new_row).T)
                final_df = final_df.append(new_row)   
                
    # Else nothing identified so look at the fragmentation data.
    else:
        #Get all the rows for this secondary ID
        print ("nothing identified here so get best match FrAnk compound")
        new_row = select_on_frank(sid_df)
        print("we are adding the row: for sid", sid)
        display(pd.DataFrame(new_row).T)
        final_df = final_df.append(new_row)  

        

# final_df = check_for_duplicates(final_df, chosen_std_cmpds) 

print ("This is the final df")
display (final_df)

print ("There are", final_df['sec_id'].nunique(), "unique compounds out of", final_df.shape[0], "rows added")



In [ ]:
def select_on_frank(sid_df):
    compound_names = sid_df['compound'].values
    name_match_dic = {}

    frank_annots = sid_df['frank_annot']
    num_att = len(frank_annots.iloc[0])
    #Look for the number of unique frank annotations -assuming each set of one annotation conatins 5 items.
    # no_attributes = (len(frank_annots[0]))
    # print (no_attributes)
    unique_f_att = set()
    for f in frank_annots:
        items = f.items()
        for i in items:
            unique_f_att.add(i)
    print ("the unique frank attribues are", unique_f_att)
    #If we have only one frank_annot i.e. all the colums have the same FrAnK Annotation returned
    if len(unique_f_att)==num_att:
        
        single_annot = frank_annots.iloc[0]
        
        for pimp_cmpd_name in compound_names:
            #Find the best fit for to frank.
            frank_cmpd_name = single_annot['frank_cmpd_name']
            m = SequenceMatcher(None, frank_cmpd_name, pimp_cmpd_name)
            name_match_dic[pimp_cmpd_name]=m.ratio()
            
            
        if name_match_dic:
            print (name_match_dic)
            max_value = max(name_match_dic.values())  # maximum value
            max_keys = [k for k, v in name_match_dic.items() if v == max_value]
            max_key = max_keys[0]
            
            
            if (max_value >= 0.5):
                print ("max_key to grab row", max_key)
         
            
                #If the match is less than 50% just take the frank annotation instead.

                #Take the first compound with this name
                cmpd_id = sid_df[sid_df['compound'] == max_key]['cmpd_id'].iloc[0]
                new_row = sid_df[sid_df['compound'] == max_key].iloc[0]   

                ucid = new_row['cmpd_id']
                cmpd_rows_df = sid_df[sid_df.cmpd_id==ucid]
                identifiers = get_all_identifiers(cmpd_rows_df)
                new_row.at['identifier'] = identifiers
            
            
            if (max_value < 0.5):
                
                print ("max_value is < 0.5 (", max_value, ") and so taking frank annot")
                
                new_row = get_frank_annot(sid_df)
            
        print ("The FrAnK probability score is", single_annot['probability'] )

   
    else:         
        print("Looks like there is more than one set of attributes for this SID, who knew this could happen??")
        sys.exit()
              
    return new_row
        

In [ ]:
def get_frank_annot(sid_df):
    
    new_row = None
    frank_annots = sid_df['frank_annot']
    single_annot = frank_annots.iloc[0] 
    
    if any(single_annot):
        
        print ("single_annot type", type(single_annot))
  
    
        ("the FrAnk annotation is: ", single_annot)
        new_row = sid_df.iloc[0]
        new_row.at['compound'] = single_annot['frank_cmpd_name']
    
    

        #Get all the frAnk identifiers for a single PiMP compound (could be one).
        identifiers=[]
        identifier_keys =['inchikey','cas_code','hmdb_id']
        for i in identifier_keys:
                identifiers.append(single_annot[i])
        new_row.at['identifier']=identifiers
    
    return new_row
    

In [ ]:
#This should be used if more than 1 standard compound has been identified for a single peak.
"""
This takes in a list of the standard compounds for a unique peak ID and returns a 
new row if a preferred standard comound can be identified. If not, None is returned.
"""
def select_standard_cmpd(sid_df, standard_cmpds):
    #For each of the standard compounds identified for the peak 
    new_row = None
    display(standard_cmpds)
    name_match_dic = {}


    for i in standard_cmpds['identifier'].unique():
        pimp_cmpd_name = standard_cmpds[standard_cmpds.identifier==i]['compound'].iloc[0]
        annotation = standard_cmpds['frank_annot'].values[0] #Get the value in the cell
        
        #If there is a FrAnK annotation get the best name match       
        if (annotation is not None):
            
            frank_cmpd_name = annotation['frank_cmpd_name']
            m = SequenceMatcher(None, frank_cmpd_name, pimp_cmpd_name)
            name_match_dic[pimp_cmpd_name]=m.ratio()
    
            print (name_match_dic)
            max_value = max(name_match_dic.values())  # maximum value
            max_keys = [k for k, v in name_match_dic.items() if v == max_value]
            max_key = max_keys[0]
            print ("max_key to grab row", max_key)


            cmpd_id = standard_cmpds[standard_cmpds['compound'] == max_key]['cmpd_id'].iloc[0]
            new_row = standard_cmpds[standard_cmpds['compound'] == max_key].iloc[0]   

            ucid = new_row['cmpd_id']
            cmpd_rows_df = sid_df[sid_df.cmpd_id==ucid]
            identifiers = get_all_identifiers(cmpd_rows_df)
            new_row.at['identifier'] = identifiers       

    print ("sending back a new row from select_standard_compound of type ", type(new_row))
    return new_row

        
    
    
    

In [ ]:
def get_peak_by_cmpd_id(sid_df, ucid): 
    
    new_row = None #Clear the new row at this stage
    cmpd_rows_df = sid_df[sid_df.cmpd_id==ucid]
    identifiers = get_all_identifiers(cmpd_rows_df)
    print ("The returned identifiers are ",identifiers)
    cmpd_id = cmpd_rows_df['cmpd_id'] == ucid
    
    #Take the row with std_db just to remember this was identified.
    db = cmpd_rows_df['db'] == 'stds_db'
    new_row = cmpd_rows_df[cmpd_id & db].iloc[0]
    new_row.at['identifier'] = identifiers
    
    print ('Returning new_row by cmpd_id')
    
    return new_row

### Check for duplicates in the final identified DF 

In [ ]:
# Duplicate peaks are ones which had more than one standard compound identified for them.

duplicate_df = final_id_df[final_id_df.sec_id.duplicated()]
display(duplicate_df)

MASS_TOL = 0.1
RT_TOL = 5

dup_ids = duplicate_df['sec_id'].values

for dupid in dup_ids:
    
    dup_peaks = final_id_df[final_id_df.sec_id==dupid]
    
    print ("duplicate peaks are: ")
    display (dup_peaks)
        
#     Assuming that all of the data for these duplicate peaks are the same, take the firt value.

    neutral_mass = dup_peaks['neutral_mass'].iloc[0]
    rt = dup_peaks['rt'].iloc[0]
    adduct = dup_peaks['adduct'].iloc[0]

    
    min_mass = neutral_mass-MASS_TOL
    max_mass = neutral_mass+MASS_TOL
   
    min_rt = rt-RT_TOL
    max_rt = rt+RT_TOL
    
    
    mass_match = final_id_df['neutral_mass'].between(min_mass, max_mass, inclusive=True)
    rt_match = final_id_df['rt'].between(min_rt, max_rt, inclusive=True)
    no_duplicates = final_id_df['sec_id'] != dupid
    
   # If we have stored another peak that within a neutral_mass and rt tolerance.
   
    matching_cmpd_df = final_id_df[mass_match & rt_match & no_duplicates]
    
    print ("Other peaks that match the duplicate peaks are:")                                  
    display(matching_cmpd_df)
    
    
    if matching_cmpd_df.index.any():
    
        print ("There are matching compounds!!!")
        #Calculate some values that may be useful.

#         matching_cmpd_df.loc[:,'abs_mdif'] = (matching_cmpd_df['neutral_mass'] - neutral_mass).abs()
#         matching_cmpd_df.loc[:,'abs_rtdif'] = (matching_cmpd_df['rt'] - rt).abs()

    #   Peak ids for the peaks that match the duplicate peak. 
        match_sec_ids = matching_cmpd_df['sec_id'].values

        for m in match_sec_ids:

#             dup_names = final_id_df[final_id_df.sec_id==dupid]['compound'].values
            
            
            dup_names = dup_peaks['compound'].values
            match_name = matching_cmpd_df[matching_cmpd_df.sec_id==m]['compound'].iloc[0]
            match_adduct = matching_cmpd_df[matching_cmpd_df.sec_id==m]['adduct'].iloc[0]
            print ("dup_names ", dup_names)
            for dup_name in dup_names:
                print (dup_name)
                keeping = False
                if dup_name == match_name:
                    print ("names are the same")
                    if match_adduct != adduct:
                        print ("adducts are different")
                        print ("keeping this:")
                        display (dup_peaks[dup_peaks['compound']==dup_name])
                        keeping = True
#                     final_id_df.index[final_id_['BoolCol'] == True].tolist()
                    else:
                        print ("ADDUCTS ARE THE SAME")
                        
                if not keeping:
                
                    print ("should delete this:")
                    display (dup_peaks[dup_peaks['compound']==dup_name])
                    index_to_drop = dup_peaks.index[dup_peaks['compound'] == dup_name].tolist()

                    print ("trying to drop index ", index_to_drop[0])
                    print ("Trying to delete")
                    display(dup_peaks.loc[index_to_drop])

                    final_id_df = final_id_df.drop(index_to_drop)


    else:
        print ("no matching compounds in the final DF")
        
        
        dup_names = dup_peaks['compound'].values
        for n in dup_names:
            m_name = final_id_df['compound']==n
            match_name = final_id_df[m_name & no_duplicates]
            print ("This peak matches on name:")
            match_name.loc[:,'abs_mdif'] = (match_name['neutral_mass'] - neutral_mass).abs()
            match_name.loc[:,'abs_rtdif'] = (match_name['rt'] - rt).abs()

            display(match_name)
        

    
    
    #Check if the mass and RT match a peak of the opposide adduct that we have already stored in the final_id_df
    
    
    
    
#     [3722 2226 1147 2589 2909 2742]
    

## Populate peaks

### For each heading the JSON files give a dictionary where the key is the unique ID/row and the value is that of the header.

In [ ]:
# Hearders returned from JSON
peak_attributes = ['sec_id','mass','rt','polarity','compound','formula','identified','identifier','frank_annot','adduct']

# These are the field names in the Peak model
peak_model_attributes = ['psec_id','m_z','rt','polarity','cmpd_name','cmpd_formula','identified','cmpd_identifiers','frank_anno','adduct']

#Dictionary to relate the JSON key names to the peak model column names.
json_model_dict = collections.OrderedDict()

for (pa, pma) in zip(peak_attributes, peak_model_attributes):
    json_model_dict[pa] = pma

print (json_model_dict)


#Dictionary that stores the Peak model attributes and values.
data_dict ={}


#For each peak in the JSON file make a new Peak object
for i in range (0,len(peak_details['pid'])):
    for attribute in json_model_dict:
       # Get the dictionary for this attribute
        current_attribute = peak_details[attribute]
        if json_model_dict[attribute]=='frank_anno':
            data_dict[json_model_dict[attribute]] = str(current_attribute[str(i)]) #Frank info from Dict to string for the moment
        else:
            data_dict[json_model_dict[attribute]] = current_attribute[str(i)]
    print(data_dict)
        
    #Add the peak to the DB
    peak_serializer = PeakSerializer(data=data_dict)
    if peak_serializer.is_valid():
        db_peak = peak_serializer.save()
        print ("peak saved ", db_peak.pid)
    else:
        print (peak_serializer.errors)


### Given a compound ID return all of the identifiers for this compound

### Given a FrAnK annotation dictionary

### Add neutral masses to the DF


In [4]:
def add_neutral_masses(selected_df):
    masses = selected_df['mass'].values
    adducts = selected_df['adduct'].values
    print ("adding neutral masses to the dataframe")
    # neutral_masses = [get_neutral_mass(mass, adduct) for mass in masses for adduct in adducts]
    neutral_masses = []
    joint_list =[masses, adducts]
    # mass_adducts = OrderedDict(zip(selected_df.mass, selected_df.adduct))

    mass_adducts = list(zip(*joint_list))
    for ma in mass_adducts:
        mass = ma[0]
        adduct = ma[1]
        neutral_mass = get_neutral_mass(mass, adduct)
        neutral_masses.append(neutral_mass) 
    print(len(neutral_masses))
    selected_df.loc[:,'neutral_mass'] = np.asarray(neutral_masses)

    return selected_df



In [5]:
#Small function to return a netral mass given the m/z and the adduct.
def get_neutral_mass(mass, adduct):
    if adduct == 'M+H':
        neutral_mass = mass-PROTON
    elif adduct == 'M-H':
        neutral_mass = mass+PROTON
    else:
        print ("What's going on?")
    
    return neutral_mass
        

### Given a peak dataframe get all the unique identifiers in the DF


In [3]:
# Takes in a df and returns a list of identifiers
def get_all_identifiers(cmpd_rows_df):

    #Take all the rows relating to a unique compound
    num_rows = cmpd_rows_df.shape[0]
    print ("the number of rows are ", num_rows)

    #Get all the identifiers for a single PiMP compound (could be one but want this as list).
    identifiers=[]
    for i in range(0,(num_rows)):
        new_id = cmpd_rows_df.iloc[i]['identifier']
        identifiers.append(new_id)

        #Take one of the rows for this compound, add identifiers and save in the final df.
    
    return identifiers